In [33]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
#     data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*2+(data['minute']/30).astype('int')
    del data['context_timestamp_tmp']
    del data['hour']
    del data['minute']

#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
#     for i in range(3):
#         temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_occupation_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
#     for i in range(3):
#         temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['property_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
#     for i in range(1,3):
#         temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
#         temp1 = temp.reset_index()
#         temp2 = temp.groupby('user_gender_id').sum().reset_index()
#         temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
#         temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#         temp3['category_%dpv'%(i)] = temp4
#         temp3.drop('instance_id_x', axis=1, inplace=True)
#         temp3.drop('instance_id_y', axis=1, inplace=True)
#         data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
#     temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
#     temp1 = temp.reset_index()
#     temp2 = temp.groupby('user_age_level').sum().reset_index()
#     temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
#     temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
#     temp3['age_pv'] = temp4
#     temp3.drop('instance_id_x', axis=1, inplace=True)
#     temp3.drop('instance_id_y', axis=1, inplace=True)
#     data = pd.merge(data,temp3,on=['user_age_level','user_age_level'],how='left')
#     del(temp3)
#     del(temp2)
#     del(temp1)
#     del(temp4)
#     del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data

In [48]:
print('train')
train = pd.read_csv('train.txt',sep=" ")
data2 = pd.read_csv('1234.csv',index_col=False) 
datac = pd.read_csv('clickpv.csv',index_col=False) 
train.insert(1, 'item_category_list_num', data2[['item_category_list']])
train.insert(2, 'item_property_list_num', data2[['item_property_list']])
train.insert(1, 'clickpv', datac[[ 'clickpv']])
train = pre_process(train)

all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
all_shape (478138, 34)
2018-09-24 23:59:47
(207904, 34)
(121035, 34)


In [49]:
# train['halfhour']

In [50]:
train.columns

Index(['instance_id', 'clickpv', 'item_category_list_num',
       'item_property_list_num', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'is_trade', 'category_0', 'category_1', 'category_2', 'property_0',
       'property_1', 'property_2', 'halfhour'],
      dtype='object')

In [51]:
print('test')
test_a = pd.read_csv('test.txt',sep=" ")
data3 = pd.read_csv('456.csv',index_col=False)
data2c = pd.read_csv('test_clickpv.csv',index_col=False) 
# data2.pop('item_category_list')
test_a.insert(1, 'item_category_list_num', data3[['item_category_list']])
# data2.pop('item_property_list')
test_a.insert(2, 'item_property_list_num', data3[['item_property_list']])
test_a.insert(1, 'clickpv', data2c[[ 'clickpv']])
print(test_a.shape)
test_a = pre_process(test_a)

test
(18371, 29)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [52]:
test_a

,instance_id,clickpv,item_category_list_num,item_property_list_num,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,...,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2,halfhour
0,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.964023,0.966550,0.955378,7908382889764677758,8277336076276184272,,2636395404473730413,7515802706813700848,302719828435032992,46
1,398316874173557226,0.01,2,0,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,34
2,6586402638209028583,0.01,2,2,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,31
3,1040996105851528465,0.01,2,1,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,33
4,6316278569655873454,0.01,2,4,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,32
5,868158305045921978,0.01,2,0,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,34
6,5713520501786699854,0.01,2,0,1171140458228333440,1547983184598012595,8072963182326625214,5,6,6,...,0.976897,0.977126,0.984791,7908382889764677758,4879721024980945592,,5131280576272319091,2636395404473730413,3163265386149801264,25
7,932945015407923184,0.01,2,1,7020520887593189887,8324857885829459863,1019055478500227370,7,10,11,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,5131280576272319091,1013419034389836991,5195139481388729954,28
8,1919197847086752313,0.01,2,0,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,24
9,304887065966615346,0.01,2,1,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,30


In [53]:
# 这里是增加的内容
import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count


In [54]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)

train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)
y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')



2018-09-20 00:00:01
2018-09-18 00:00:01
1 days 23:59:54
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43


In [55]:
print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data



(18371, 34)


In [56]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
feat_set = list(test_a.columns)
for i,feat in enumerate(feat_set):
    tmp = lb.fit_transform((list(train[feat])+list(val[feat])+list(test_a[feat])))
    print(tmp)
    enc.fit(tmp.reshape(-1,1))
    x_train = enc.transform(lb.transform(train[feat]).reshape(-1, 1))
    x_test = enc.transform(lb.transform(test_a[feat]).reshape(-1, 1))
    x_val = enc.transform(lb.transform(val[feat]).reshape(-1, 1))
    if i == 0:
        X_train, X_test,X_val = x_train, x_test,x_val
    else:
        X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))




baseline ing ... ...
Index(['clickpv', 'item_category_list_num', 'item_property_list_num',
       'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
       'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_id', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'category_0', 'category_1', 'category_2', 'property_0', 'property_1',
       'property_2', 'halfhour', 'user_count', 'item_count'],
      dtype='object')
[0 0 0 ..., 0 3 0]
[2 2 2 ..., 2 2 2]
[2 1 0 ..., 0 0 1]
[3411  309 5195 ..., 1855 1855 1855]
[ 418 1895 1182 ..., 1058 1058 1058]
[50  9  9 ..., 50 50 50]
[3 8 8 ..., 6 6 6]
[ 4  9  9 ..., 10 10 10]
[ 4  8 10 ..., 12 12 12]
[14 14 16 ..., 11 11 11]
[142934 132470  27030 ...,  95716 115737 108386]
[1 1 1

In [57]:
lr = LogisticRegression()


lr.fit(X_train, y_train)
proba_val = lr.predict_proba(X_val)[:,1]
proba_sub = lr.predict_proba(X_test)[:,1]

In [58]:
print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print(log_loss(y_val,proba_val))

0.0496218133388
0.0850677989128


In [59]:
X_train.shape

(207904, 565154)

In [69]:
%%time
import lightgbm as lgb

gbm = lgb.LGBMClassifier(objective='binary',
                        num_leaves=64,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.75,
                        gpu_platform_id = 0, gpu_device_id = 0
                        ,reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)

[1]	valid_0's binary_logloss: 0.683888
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674828
[3]	valid_0's binary_logloss: 0.665936
[4]	valid_0's binary_logloss: 0.657214
[5]	valid_0's binary_logloss: 0.648656
[6]	valid_0's binary_logloss: 0.640268
[7]	valid_0's binary_logloss: 0.632036
[8]	valid_0's binary_logloss: 0.623957
[9]	valid_0's binary_logloss: 0.616031
[10]	valid_0's binary_logloss: 0.608258
[11]	valid_0's binary_logloss: 0.600621
[12]	valid_0's binary_logloss: 0.593121
[13]	valid_0's binary_logloss: 0.585762
[14]	valid_0's binary_logloss: 0.578529
[15]	valid_0's binary_logloss: 0.571425
[16]	valid_0's binary_logloss: 0.564456
[17]	valid_0's binary_logloss: 0.557613
[18]	valid_0's binary_logloss: 0.55088
[19]	valid_0's binary_logloss: 0.544259
[20]	valid_0's binary_logloss: 0.537761
[21]	valid_0's binary_logloss: 0.531372
[22]	valid_0's binary_logloss: 0.525088
[23]	valid_0's binary_logloss: 0.518914
[24]	valid_0's binary_loglo

[204]	valid_0's binary_logloss: 0.124944
[205]	valid_0's binary_logloss: 0.124408
[206]	valid_0's binary_logloss: 0.123874
[207]	valid_0's binary_logloss: 0.123347
[208]	valid_0's binary_logloss: 0.122826
[209]	valid_0's binary_logloss: 0.122314
[210]	valid_0's binary_logloss: 0.121805
[211]	valid_0's binary_logloss: 0.121303
[212]	valid_0's binary_logloss: 0.120806
[213]	valid_0's binary_logloss: 0.120317
[214]	valid_0's binary_logloss: 0.119828
[215]	valid_0's binary_logloss: 0.11935
[216]	valid_0's binary_logloss: 0.118877
[217]	valid_0's binary_logloss: 0.118411
[218]	valid_0's binary_logloss: 0.117951
[219]	valid_0's binary_logloss: 0.117494
[220]	valid_0's binary_logloss: 0.117044
[221]	valid_0's binary_logloss: 0.116599
[222]	valid_0's binary_logloss: 0.11616
[223]	valid_0's binary_logloss: 0.115725
[224]	valid_0's binary_logloss: 0.115301
[225]	valid_0's binary_logloss: 0.114879
[226]	valid_0's binary_logloss: 0.114462
[227]	valid_0's binary_logloss: 0.114055
[228]	valid_0's bi

[404]	valid_0's binary_logloss: 0.086075
[405]	valid_0's binary_logloss: 0.0860364
[406]	valid_0's binary_logloss: 0.0860015
[407]	valid_0's binary_logloss: 0.0859659
[408]	valid_0's binary_logloss: 0.0859292
[409]	valid_0's binary_logloss: 0.0858915
[410]	valid_0's binary_logloss: 0.0858573
[411]	valid_0's binary_logloss: 0.0858241
[412]	valid_0's binary_logloss: 0.0857898
[413]	valid_0's binary_logloss: 0.0857572
[414]	valid_0's binary_logloss: 0.0857241
[415]	valid_0's binary_logloss: 0.0856951
[416]	valid_0's binary_logloss: 0.0856637
[417]	valid_0's binary_logloss: 0.0856304
[418]	valid_0's binary_logloss: 0.0855984
[419]	valid_0's binary_logloss: 0.085569
[420]	valid_0's binary_logloss: 0.0855395
[421]	valid_0's binary_logloss: 0.0855099
[422]	valid_0's binary_logloss: 0.0854818
[423]	valid_0's binary_logloss: 0.0854546
[424]	valid_0's binary_logloss: 0.0854276
[425]	valid_0's binary_logloss: 0.0853993
[426]	valid_0's binary_logloss: 0.0853735
[427]	valid_0's binary_logloss: 0.08

[601]	valid_0's binary_logloss: 0.0835162
[602]	valid_0's binary_logloss: 0.0835142
[603]	valid_0's binary_logloss: 0.0835101
[604]	valid_0's binary_logloss: 0.0835053
[605]	valid_0's binary_logloss: 0.0835007
[606]	valid_0's binary_logloss: 0.0834957
[607]	valid_0's binary_logloss: 0.083492
[608]	valid_0's binary_logloss: 0.0834882
[609]	valid_0's binary_logloss: 0.083485
[610]	valid_0's binary_logloss: 0.0834821
[611]	valid_0's binary_logloss: 0.0834793
[612]	valid_0's binary_logloss: 0.083477
[613]	valid_0's binary_logloss: 0.0834743
[614]	valid_0's binary_logloss: 0.0834692
[615]	valid_0's binary_logloss: 0.0834666
[616]	valid_0's binary_logloss: 0.0834633
[617]	valid_0's binary_logloss: 0.0834599
[618]	valid_0's binary_logloss: 0.083457
[619]	valid_0's binary_logloss: 0.0834564
[620]	valid_0's binary_logloss: 0.0834541
[621]	valid_0's binary_logloss: 0.0834486
[622]	valid_0's binary_logloss: 0.0834454
[623]	valid_0's binary_logloss: 0.0834424
[624]	valid_0's binary_logloss: 0.0834

[800]	valid_0's binary_logloss: 0.0831792
[801]	valid_0's binary_logloss: 0.0831782
[802]	valid_0's binary_logloss: 0.0831797
[803]	valid_0's binary_logloss: 0.0831795
[804]	valid_0's binary_logloss: 0.0831788
[805]	valid_0's binary_logloss: 0.0831774
[806]	valid_0's binary_logloss: 0.0831774
[807]	valid_0's binary_logloss: 0.0831769
[808]	valid_0's binary_logloss: 0.0831772
[809]	valid_0's binary_logloss: 0.0831765
[810]	valid_0's binary_logloss: 0.0831751
[811]	valid_0's binary_logloss: 0.0831725
[812]	valid_0's binary_logloss: 0.0831698
[813]	valid_0's binary_logloss: 0.0831699
[814]	valid_0's binary_logloss: 0.0831675
[815]	valid_0's binary_logloss: 0.0831665
[816]	valid_0's binary_logloss: 0.083167
[817]	valid_0's binary_logloss: 0.0831653
[818]	valid_0's binary_logloss: 0.0831652
[819]	valid_0's binary_logloss: 0.0831643
[820]	valid_0's binary_logloss: 0.083163
[821]	valid_0's binary_logloss: 0.0831635
[822]	valid_0's binary_logloss: 0.0831627
[823]	valid_0's binary_logloss: 0.08

[999]	valid_0's binary_logloss: 0.0831188
[1000]	valid_0's binary_logloss: 0.0831194
[1001]	valid_0's binary_logloss: 0.083119
[1002]	valid_0's binary_logloss: 0.0831199
[1003]	valid_0's binary_logloss: 0.0831195
[1004]	valid_0's binary_logloss: 0.08312
[1005]	valid_0's binary_logloss: 0.0831198
[1006]	valid_0's binary_logloss: 0.0831193
[1007]	valid_0's binary_logloss: 0.0831192
[1008]	valid_0's binary_logloss: 0.0831194
[1009]	valid_0's binary_logloss: 0.0831198
[1010]	valid_0's binary_logloss: 0.0831196
[1011]	valid_0's binary_logloss: 0.0831187
[1012]	valid_0's binary_logloss: 0.0831177
[1013]	valid_0's binary_logloss: 0.0831171
[1014]	valid_0's binary_logloss: 0.0831166
[1015]	valid_0's binary_logloss: 0.083116
[1016]	valid_0's binary_logloss: 0.0831156
[1017]	valid_0's binary_logloss: 0.0831153
[1018]	valid_0's binary_logloss: 0.083114
[1019]	valid_0's binary_logloss: 0.0831145
[1020]	valid_0's binary_logloss: 0.0831151
[1021]	valid_0's binary_logloss: 0.0831142
[1022]	valid_0's 

[1191]	valid_0's binary_logloss: 0.0831306
[1192]	valid_0's binary_logloss: 0.0831309
[1193]	valid_0's binary_logloss: 0.0831317
[1194]	valid_0's binary_logloss: 0.0831311
[1195]	valid_0's binary_logloss: 0.0831298
[1196]	valid_0's binary_logloss: 0.0831287
[1197]	valid_0's binary_logloss: 0.0831283
[1198]	valid_0's binary_logloss: 0.0831298
[1199]	valid_0's binary_logloss: 0.0831317
[1200]	valid_0's binary_logloss: 0.0831313
[1201]	valid_0's binary_logloss: 0.0831316
[1202]	valid_0's binary_logloss: 0.0831315
[1203]	valid_0's binary_logloss: 0.0831316
[1204]	valid_0's binary_logloss: 0.0831309
[1205]	valid_0's binary_logloss: 0.083131
[1206]	valid_0's binary_logloss: 0.0831314
[1207]	valid_0's binary_logloss: 0.083131
[1208]	valid_0's binary_logloss: 0.0831301
[1209]	valid_0's binary_logloss: 0.0831299
[1210]	valid_0's binary_logloss: 0.0831315
Early stopping, best iteration is:
[1060]	valid_0's binary_logloss: 0.083106
Wall time: 1min 3s


In [70]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict_proba(X_val, num_iteration=gbm.best_iteration_ )[:,1]
y_sub_1 = gbm.predict_proba(X_test, num_iteration=gbm.best_iteration_ )[:,1]
print()
print(log_loss(y_train,gbm.predict_proba(X_train)[:,1]))
print(log_loss(y_val,y_pred_1))

print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0831080714914

Start predicting...


D:\anaconda\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')



0.0816848393185
0.0831059948957
0.0832182908374


In [17]:
%%time
import xgboost as xgb

model_xgb = xgb.XGBClassifier( n_estimators=500,random_state =666, nthread = -1,n_jobs=-1)
model_xgb.fit(X_train, y_train,eval_metric='logloss')
# param['gpu_id'] = 0
# param['tree_method'] = 'gpu_hist' # or 'gpu_exact'

y_xpred_1 = model_xgb.predict_proba(X_val)[:,1]
y_xsub_1 = model_xgb.predict_proba(X_test)[:,1]


Wall time: 3min 13s


In [18]:
print(log_loss(y_train,model_xgb.predict_proba(X_train)))
print(log_loss(y_val, y_xpred_1))

0.0868091590221
0.0832891431902


In [19]:
print(log_loss(y_val,(y_pred_1 + y_xpred_1)/2))

0.0831656582457


In [20]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import dok_matrix

In [22]:
ENet = make_pipeline(ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=40, max_features='sqrt',
                                   min_samples_leaf=150, min_samples_split=100, 
                                   loss='huber', random_state =5)

In [30]:
ENet.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('elasticnet', ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=3, selection='cyclic', tol=0.0001, warm_start=False))])

In [34]:
epre = ENet.predict(X_val)

In [38]:
log_loss(y_val,epre)

0.085058479094694939

In [ ]:
log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes)

In [ ]:
GBoost.fit(X_train, y_train)

In [ ]:
import numpy as np
bes = 1
for i in np.arange(0,1,0.01):
    if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
        bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
print(bes)#0.083131236937

In [293]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)

In [21]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list((y_sub_1 + y_xsub_1)/2)
sub.to_csv('20180327.txt',sep=" ",index=False)

In [ ]:
(proba_sub + y_sub_1)/2